In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

Importando os dados de treino e teste:

In [ ]:
treino = pd.read_csv("../input/atividade-regressao-PMR3508/train.csv", index_col='Id')
teste_x = pd.read_csv("../input/atividade-regressao-PMR3508/test.csv", index_col='Id')
treino_y = treino['median_house_value']
treino_x=pd.read_csv("../input/atividade-regressao-PMR3508/train.csv", index_col='Id')
treino_x.drop(['median_house_value'], axis = 1, inplace = True)

# Data preparation:

Checando as covariaveis existentes do problema:

In [ ]:
treino.head()

Verificando se há missing data:(não há)

In [ ]:
treino.info()

Para ter uma noção de qual região estamos tratando, os dados foram mapeados.Se observa que é uma região litorânea, portanto a altitude (distância com o mar) seria uma covariável interessante de se ter em mãos. No entanto, ela não foi considerada nesse projeto. Seria interessante de posse de um mapa criar alguma covariável nova. É evidente que o valor de uma casa depende fortemente da distância também até um centro urbano grande na região; olhando no mapa defini três centros mais importantes: São Francisco, São Diego e Los Angeles. Assim, foi criada uma nova covariável que é a distância ate algum desses centros.

In [ ]:
import plotly.express as px

mapa = px.scatter_mapbox(treino, lat="latitude", lon="longitude", color="median_house_value", size="population",
                  color_continuous_scale=px.colors.sequential.Sunsetdark, size_max=15, zoom=4,
                  mapbox_style="carto-positron")
mapa.show()

In [ ]:
from geopy.distance import geodesic
LA=(34.0194,-118.411)
SD=(37.335480,-117.135)
SF=(37.7272,-123.032)
def distance_cidade(x):
    coordenada=(x["latitude"],x["longitude"])
    x["distancia_cidade"] = np.sqrt(min(geodesic(coordenada, LA).km, geodesic(coordenada, SF).km,geodesic(coordenada, SD).km))
    
    return (x)
treino_x = treino_x.apply(distance_cidade, axis = 1)
treino = treino.apply(distance_cidade, axis = 1)
teste_x = teste_x.apply(distance_cidade, axis = 1)

In [ ]:
treino_x.head()

Observa-se a correlação entre os atributos:

In [ ]:
from sklearn import preprocessing
LabelEncoder = preprocessing.LabelEncoder()
correlationMap = treino.apply(preprocessing.LabelEncoder().fit_transform).corr()
plt.figure(figsize=(10,7))
sns.heatmap(treino.corr(method="pearson"), square=True, annot=True, cbar=True, vmin=-1, vmax=1, cmap="Greys")

Observa-se que o preço de uma casa depende basicamente de três atributos, distância até um centro urbano próximo, a renda e um pouco da latitude. Seria interessante de alguma juntar as influências de outras covariáveis menos influentes como número de casa, total de quartos e população em variaveis mais influentes. Assim criou-se novas variáveis: 

In [ ]:
treino_x['total_rooms/households']=treino_x['total_rooms']/treino_x['households']
treino_x['population/households']=treino_x['population']/treino_x['households']
treino_x['total_bedrooms/total_rooms']=treino_x['total_bedrooms']/treino_x['total_rooms']
treino['total_rooms/households']=treino['total_rooms']/treino['households']
treino['population/households']=treino['population']/treino['households']
treino['total_bedrooms/total_rooms']=treino['total_bedrooms']/treino['total_rooms']
teste_x['total_rooms/households']=teste_x['total_rooms']/teste_x['households']
teste_x['population/households']=teste_x['population']/teste_x['households']
teste_x['total_bedrooms/total_rooms']=teste_x['total_bedrooms']/teste_x['total_rooms']

Observa-se a correlação entre os atributos novamente:

In [ ]:
from sklearn import preprocessing
LabelEncoder = preprocessing.LabelEncoder()
correlationMap = treino.apply(preprocessing.LabelEncoder().fit_transform).corr()
plt.figure(figsize=(10,7))
sns.heatmap(treino.corr(method="pearson"), square=True, annot=True, cbar=True, vmin=-1, vmax=1, cmap="Greys")

Retirando as covariáveis menos importantes agora que temos covariáveis melhores.

In [ ]:
treino_x = treino_x.drop(["latitude", "longitude"], axis = 1)
teste_x = teste_x.drop(["latitude", "longitude"], axis = 1)
treino_x = treino_x.drop(["total_bedrooms","households","population","total_rooms"], axis = 1)
teste_x = teste_x.drop(["total_bedrooms","households","population","total_rooms"], axis = 1)

# Regressão simples:

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
reg_s = linear_model.LinearRegression()

Observação do parâmetro R2:

In [ ]:
r2_scores=cross_val_score(reg_s,treino_x,treino['median_house_value'],cv=10,scoring='r2')
r2_scores.mean()

Treinando o regressor:

In [ ]:
reg_s.fit(treino_x,treino['median_house_value'])

# Regressão Ridge:

É importante normalizar os dados para realizar a regressão rigde e lasso a seguir.

In [ ]:
import scipy.stats as stats
stats.zscore(treino_x)

Definindo o hiper-parâmetro por Cross Validation:

In [ ]:
maior_média=0
for K in range(1,1000):
    reg = linear_model.Ridge(alpha=K*0.001)
    r2_scores = cross_val_score(reg,treino_x, treino['median_house_value'],cv=10,scoring='r2')
    média=r2_scores.mean()
    if(média>maior_média):
        maior_média=média
        melhor_k=K

In [ ]:
melhor_k

Houve uma melhoria pequena no parâmetro R2

In [ ]:
maior_média

Treinando o regressor:

In [ ]:
reg_r = linear_model.Ridge(alpha=melhor_k*0.001)
reg_r.fit(treino_x,treino['median_house_value'])

# Regressão Lasso:

Definindo o hiper-parâmetro por Cross Validation:

In [ ]:
maior_média=0
for K in range(1,400):
    reg = linear_model.Lasso(alpha=K*0.1)
    r2_scores = cross_val_score(reg,treino_x, treino['median_house_value'],cv=10)
    média=r2_scores.mean()
    if(média>maior_média):
        maior_média=média
        melhor_k=K

In [ ]:
melhor_k

In [ ]:
maior_média

Treinando o regressor:

In [ ]:
reg_l = linear_model.Lasso(alpha=melhor_k*0.1)
reg_l.fit(treino_x,treino['median_house_value'])

# Submissão:

In [ ]:
teste_x.index

In [ ]:
predição_simples = reg_s.predict(teste_x)
predição_ridge = reg_r.predict(teste_x)
predição_lasso = reg_l.predict(teste_x)

In [ ]:
submiss = pd.DataFrame(teste_x.index)
submiss['median_house_value']=predição_ridge
submiss.to_csv("PMR3508-2020-90_submissao.csv", index = False, index_label = 'Id')

In [ ]:
submiss.describe

Para chegar no resultado final foram sendo retiradas covariáveis aos poucos para análisar o compromisso entre critério e variância, tentando chegar empiricamente num equilibrio entre viés que é baixo com todas covariáveis envolvidas mas com variância alta. Vale notar que o parâmetro r2 foi ficando cada vez pior com menos covariáveis já que cada vez mais se distanciava de um possível sobreajuste.

Os scores foram dispostos com os índices a seguir. O melhor resultado foi usando regressão ridge com índice 4. Os passos de 0 a 8 foram feitos em ordem, partindo do passo anterior, por exemplo no passo 2 está implícito que 1 foi feito. Observa-se que a patir do passo 5 chegou-se num viés alto demais e no passo 8 voltou-se num bom viés mas que retirar a covariável median_age não vale a pena como total_rooms.

0: com todas covariáveis
1: retirou-se latitude e longitude
2: retirou-se bedroom
3: retirou-se households e population
4: retirou-se total_rooms
5: retirou-se median_age
6: acrescentou-se median_age e median_age/total_rooms
7: retirou-se median_age
8: passo 0,1,2 e 4 mas em 4 retirou-se median_age ao invés de total_rooms

s: regressão simples
r: regressão ridge
l: regressão lasso

In [ ]:
score_s_0=0.97339
score_r_0=0.98242
score_l_0=0.98195 
score_s_1=0.68243
score_r_1=0.68078
score_l_1=0.68090
score_s_2=0.67602 
score_r_2=0.67606
score_l_2=0.67604
score_s_3=0.48367
score_r_3=0.48362
score_l_3=0.48367
score_s_4=0.47571
score_r_4=0.47568
score_l_4=0.47571
score_s_5=0.47645
score_s_6=0.50099
score_s_7=0.52411
score_s_8=0.47897